In [8]:
from CreateDocuments import load_documents
%load_ext autoreload
%autoreload 2

## HuggingFace Embeddings

In [3]:
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
# Equivalent to SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
text = "This is a test document."
query_result = embeddings.embed_query(text)
query_result = embeddings.embed_query(text)
print(len(query_result))
doc_result = embeddings.embed_documents([text, "Document 2", "Document 3"])
print(len(doc_result))

384
3


## with Chroma
Need to fix text loader

In [14]:
# https://python.langchain.com/docs/integrations/vectorstores/chroma/

from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import CharacterTextSplitter

# # load the document and split it into chunks
# loader = TextLoader("../../modules/state_of_the_union.txt")
# documents = loader.load()

# # split it into chunks
# text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
# docs = text_splitter.split_documents(documents)

docs = load_documents('documents/train_size_200_overlap_20_documents.pkl')
# create the open-source embedding function
embedding_function = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# load it into Chroma
db = Chroma.from_documents(docs, embedding_function)

In [22]:

# query it
query = "When was the house at 3524 Redwing Ct, Naperville, IL 60564 last sold and for what price?"
docs = db.similarity_search(query)

# print results
print(docs[0].page_content)

3524 Redwing Ct, Naperville, IL 60564
4 beds5 baths3,591 sqft Edit


"3524 Redwing Ct, Naperville, IL 60564
4 beds5 baths3,591 sqft Edit
A Zestimate¬Æ home valuation is Zillow's estimated market value. It is not an appraisal. Use it as a starting point to determine a home's value. Learn more
Facts
- Single Family
- Built in 2000
- Views: 773 all time views
- Cooling: Central, Other
- Heating: Forced air, Other
- Last sold: Oct 2013 for $595,000
- Last sale price/sqft: $166
Features
- Ceiling Fan
- Deck
- Fireplace
- Flooring: Carpet, Hardwood
- Mother-in-Law
- Parking: Garage - Attached, 3 spaces, 704 sqft
- Security System
- Vaulted Ceiling
Appliances Included
- Dishwasher
- Dryer
- Garbage disposal
- Microwave
- Range / Oven
- Refrigerator
- Washer
Room Types
- Dining room
- Family room
- Office
- Recreation room
Construction
- Exterior material: Brick
- Roof type: Asphalt
- Room count: 11
- Stories: 2
- Structure type: Other
- Unit count: 1
Other
- Floor size: 3,591 sqft
- Heating: Gas
- Laundry: In Unit
- Parcel #: 0701093130290000
- Zillow Home ID: 556842K.
Mortgages
Neighborhood
Market guideZillow predicts 60564 home values will fall 1% next year, compared to a 1.1% decrease for Naperville as a whole. Among 60564 homes, this home is valued 49.3% more than the midpoint (median) home, and is valued 11.5% more per square.
Learn more about forecast calculations or 60564 home values.‚Ä¶ More Less
For Sale
- 3540 Redwing Ct5 beds, 5 baths
3,986 sqft, 6,372 sqft lot, built in 2004
- 3459 Redwing Dr4 beds, 3.5 baths
2,865 sqft, 10,001 sqft lot, built in 2001
- 3451 Redwing Dr5 beds, 5 baths
3,553 sqft, 10,890 sqft lot, built in 2003
- 3312 Danlaur Ct4 beds, 3.5 baths
4,410 sqft, 12,196 sqft lot, built in 2005
- 3727 Nicanoa Ln5 beds, 4.5 baths
3,700 sqft, 11,003 sqft lot, built in 2003
- 3508 Tall Grass Dr5 beds, 3.5 baths
3,405 sqft, 8,751 sqft lot, built in 2000
- 24431 Davids Ct4 beds, 2 baths
1,371 sqft, 21,000 sqft lot, built in 1972
- 3204 Rollingridge Rd4 beds, 3.5 baths
4,103 sqft, 9,147 sqft lot, built in 2004
- 3511 Kestral Dr5 beds, 3.5 baths
3,524 sqft, 10,890 sqft lot, built in 2003
- 3716 Nicanoa Ln6 beds, 4.5 baths
4,304 sqft, 4,051 sqft lot, built in 2004
- 3607 Hector Ln5 beds, 4.5 baths
4,205 sqft, 11,325 sqft lot, built in 2003
- 3416 Sunnyside Ct5 beds, 4.1 baths
3,500 sqft, 14,374 sqft lot, built in 2001
- 3408 Sunnyside Ct5 beds, 4.5 baths
3,503 sqft, 14,374 sqft lot, built in 2000
- 24337 W Boulevard De John3 beds, 2 baths
1,734 sqft, 22,651 sqft lot, built in 1974
- 3304 Mistflower Ln4 beds, 5 baths
3,114 sqft, 10,123 sqft lot, built in 2003
Rentals
- 24330 W Boulevard De John3 beds, 2 baths
-- sqft, -- sqft lot, built in --
- 3412 Minito Ct5 beds, 5 baths
4,950 sqft, -- sqft lot, built in 2005
- 3007 Saganashkee Ln2 beds, 3 baths
1,850 sqft, -- sqft lot, built in 2003
- 3007 Saganashkee Ln # 30072 beds, 2.5 baths
1,850 sqft, -- sqft lot, built in 2003
- 3443 Breitwieser Ln4 beds, 3.5 baths
3,118 sqft, 10,100 sqft lot, built in 2001
- 2809 Wheatland Ct2 beds, 2.5 baths
1,550 sqft, -- sqft lot, built in 1998
- 2817 Ledgestone Ct3 beds, 2.5 baths
1,500 sqft, -- sqft lot, built in 1998
- 2812 Cantore Rd3 beds, 2.5 baths
1,502 sqft, -- sqft lot, built in 1997
- 2876 Cedar Glade Dr3 beds, 2.5 baths
1,624 sqft, -- sqft lot, built in 1997
- 11336 Highland Dr S4 beds, 2.5 baths
2,200 sqft, -- sqft lot, built in 2005
- 11338 Highland Dr S4 beds, 2.5 baths
2,200 sqft, -- sqft lot, built in 2005
- 24008 W Marathon Ln4 beds, 2.5 baths
2,400 sqft, -- sqft lot, built in 2001
- 2818 Cedar Glade Dr # 28183 beds, 2.5 baths
1,502 sqft, -- sqft lot, built in 1997
- 2818 Cedar Glade Dr3 beds, 3 baths
1,502 sqft, -- sqft lot, built in 1997
- 2710 Idaho Rd3 beds, 2.5 baths
1,450.6 mi
- Scullen Middle (assigned)6-80.3 mi
- Waubonsie Valley."

## Jina

In [1]:
# from langchain_community.embeddings import JinaEmbeddings
# embeddings = JinaEmbeddings(
#     jina_api_key="jina_b6dea7a5e4074b9683405c67845ecc59hdfFSEyD6cdnVkhicyNhDr-WPKFh", model_name="jina-embeddings-v2-base-en"
# )
# text = "This is a test document."
# query_result = embeddings.embed_query(text)
# print(query_result)
# doc_result = embeddings.embed_documents([text])
# print(query_result)